영화 추천 엔진
 + 사용자 기반
  + 유사 선호도 사용자 발결
  + 활성 사용자에게 아이템 추천
 + 아이템 기반
  + 아이템끼리 유사도 계산하여 추천
  + 아직 평가하지 않은 아이템 찾아서 추천

모델의 평가
 + 평균 제곱 오차 MSE
 + 평균 제곱근 오차 RMSE

In [1]:
import pandas as pd

In [33]:
df = pd.read_csv('u.data', sep = "\t", header = None)
df.columns = ['user_id','item_id','rating','timestamp']

In [34]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [35]:
df.shape

(100000, 4)

In [37]:
n_user = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(f"n_user = {n_user}, n_items = {n_items}")


n_user = 943, n_items = 1682


In [39]:
import numpy as np

In [41]:
ratings = np.zeros((n_user, n_items))

In [43]:
ratings

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [42]:
ratings.shape

(943, 1682)

In [44]:
for row in df.itertuples():
    ratings[row[1]-1, row[2]-1] = row[3]

In [45]:
ratings

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [47]:
from sklearn.model_selection import train_test_split

ratings_train, ratings_test = train_test_split(ratings, test_size = 0.33, random_state= 42)

In [50]:
ratings_train.shape # 영화, 평점 정보

(631, 1682)

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

user_distances = cosine_similarity(ratings_train)

user_distances

array([[1.        , 0.36475764, 0.44246231, ..., 0.02010641, 0.33107929,
        0.25638518],
       [0.36475764, 1.        , 0.42635255, ..., 0.06694419, 0.27339314,
        0.22337268],
       [0.44246231, 0.42635255, 1.        , ..., 0.06675756, 0.25424373,
        0.22320126],
       ...,
       [0.02010641, 0.06694419, 0.06675756, ..., 1.        , 0.04853428,
        0.05142508],
       [0.33107929, 0.27339314, 0.25424373, ..., 0.04853428, 1.        ,
        0.1198022 ],
       [0.25638518, 0.22337268, 0.22320126, ..., 0.05142508, 0.1198022 ,
        1.        ]])

In [52]:
user_pred = user_distances.dot(ratings_train) / np.array([np.abs(user_distances).sum(axis = 1)]).T

In [53]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

In [55]:
np.sqrt(get_mse(user_pred, ratings_train))

2.8075245308903365

In [56]:
np.sqrt(get_mse(user_pred, ratings_test))

2.9870546415652575

In [57]:
from sklearn.neighbors import NearestNeighbors

k = 5
neigh = NearestNeighbors(n_neighbors=k, metric="cosine")

In [58]:
neigh.fit(ratings_train)

NearestNeighbors(metric='cosine')

In [61]:
top_k_distances, top_k_users = neigh.kneighbors(ratings_train, return_distance=True)

In [60]:
top_k_users

array([[  0, 589, 155,  33, 364],
       [  1, 483, 339, 172, 188],
       [  2, 382, 560, 350, 155],
       ...,
       [628, 258, 242, 229, 494],
       [629, 378, 155, 589, 591],
       [630, 495, 201, 417, 603]], dtype=int64)

In [62]:
top_k_distances

array([[0.        , 0.38230161, 0.39990633, 0.40834169, 0.4100445 ],
       [0.        , 0.4625691 , 0.50677921, 0.50811827, 0.50882566],
       [0.        , 0.46538829, 0.48267976, 0.49176259, 0.49265099],
       ...,
       [0.        , 0.5764934 , 0.59340849, 0.64699606, 0.66472075],
       [0.        , 0.60496802, 0.6115226 , 0.62054374, 0.6229481 ],
       [0.        , 0.56320216, 0.60221688, 0.60314589, 0.6400121 ]])

In [63]:
user_pred_k = np.zeros(ratings_train.shape)

In [67]:
for i in range(ratings_train.shape[0]):
    user_pred_k[i,:] = top_k_distances[i].T.dot(ratings_train[top_k_users][i]) /\
                        np.array([np.abs(top_k_distances[i].T).sum(axis= 0)]).T

In [68]:
user_pred_k

array([[4.25618269, 2.49082621, 0.71654943, ..., 0.        , 0.        ,
        0.        ],
       [3.74418756, 0.        , 2.48873124, ..., 0.        , 0.        ,
        0.        ],
       [3.22293592, 2.98635211, 2.47648118, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.07143091, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [3.73945823, 2.48622549, 1.76969702, ..., 0.        , 0.        ,
        0.        ],
       [1.95357502, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [70]:
np.sqrt(get_mse(user_pred_k, ratings_train))

2.0922014531938316

In [71]:
np.sqrt(get_mse(user_pred_k, ratings_test))

3.054698791142718